# UNIFICAR INGRESOS Y EGRESOS

In [1]:
import sys
from pathlib import Path
import pandas as pd
import re
import pandas as pd
import numpy as np
import unicodedata




# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent.parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import ARCHIVO_INGRESOS_FINAL_AJUSTADO_PATH, ARCHIVO_EGRESOS_FINAL_AJUSTADO_PATH

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Lector robusto: BOM / encoding ---
def _leer(path: Path) -> pd.DataFrame:
    path = Path(path)
    if path.suffix.lower() in {'.xlsx', '.xls'}:
        return pd.read_excel(path)
    for enc in ("utf-8-sig", "latin-1", "utf-8"):
        try:
            return pd.read_csv(path, sep=';', decimal=',', encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, sep=';', decimal=',')

# --- Repara CSV en una sola columna con ';' ---
def _reparar_unacol(df: pd.DataFrame) -> pd.DataFrame:
    if df.shape[1] == 1 and isinstance(df.columns[0], str) and df.columns[0].count(';') > 0:
        header = str(df.iloc[0, 0]).split(';')
        body = df.iloc[1:, 0].astype(str).str.split(';', expand=True)
        body.columns = [h.strip() for h in header]
        return body.reset_index(drop=True)
    return df

# --- Coalesce de variantes de nombre para cada campo requerido ---
def _coalesce_cols(df: pd.DataFrame, variantes: list[str], as_datetime=False, dayfirst=True):
    existentes = [c for c in variantes if c in df.columns]
    if not existentes:
        return pd.Series(np.nan, index=df.index)
    tmp = df[existentes].copy()
    s = tmp.bfill(axis=1).iloc[:, 0]
    if as_datetime:
        s = pd.to_datetime(s, errors='coerce', dayfirst=dayfirst)
    return s

def _prep_5cols_con_tipo(path: Path, tipo: str) -> pd.DataFrame:
    df = _reparar_unacol(_leer(path))

    FECHA_VARS = ['ï»¿FechaEmision','FechaEmision','Fecha Emisión','Fecha Emision','fecha_emision','Fecha']
    ANIO_VARS  = ['Año','AÃ±o','Anio','anio','año']
    MES_VARS   = ['Mes','mes']
    IMPNGT_VARS = ['Imp. Neto Gravado Total','Imp. Neto Gravado  Total',
                   'Imp_neto_gravado_total','Imp. Neto Gravado IVA Total']
    TVNOM_VARS = ['TotalVenta_Real','Total Venta Real','totalventareal']

    out = pd.DataFrame({
        'FechaEmision': _coalesce_cols(df, FECHA_VARS, as_datetime=False),
        'Año':           _coalesce_cols(df, ANIO_VARS),
        'Mes':           _coalesce_cols(df, MES_VARS),
        'Imp. Neto Gravado Total': _coalesce_cols(df, IMPNGT_VARS),
        'TotalVenta_Real':      _coalesce_cols(df, TVNOM_VARS),
    })
    out['Tipo'] = tipo  # "Ingreso" o "Egreso"
    return out

# --- Construcción, orden y export ---
ing = _prep_5cols_con_tipo(ARCHIVO_INGRESOS_FINAL_AJUSTADO_PATH, 'Ingreso')
egr = _prep_5cols_con_tipo(ARCHIVO_EGRESOS_FINAL_AJUSTADO_PATH,  'Egreso')

unificado = pd.concat([ing, egr], ignore_index=True)

# Ordenar por fecha (sin cambiar el texto original), luego Año/Mes, y que Ingreso aparezca antes que Egreso en misma fecha
_fecha_tmp = pd.to_datetime(unificado['FechaEmision'], errors='coerce', dayfirst=True)
_tipo_rank = unificado['Tipo'].map({'Ingreso': 0, 'Egreso': 1})
unificado = (unificado
             .assign(_fecha_tmp=_fecha_tmp, _tipo_rank=_tipo_rank)
             .sort_values(['_fecha_tmp','Año','Mes','_tipo_rank'], na_position='last')
             .drop(columns=['_fecha_tmp','_tipo_rank'])
             .reset_index(drop=True))

# Exportar (mismas 5 columnas + Tipo)
cols_finales = ['FechaEmision','Año','Mes','Imp. Neto Gravado Total','TotalVenta','Tipo']
out_dir = Path(ARCHIVO_INGRESOS_FINAL_AJUSTADO_PATH).parent
out_path = out_dir / "facturacion_total_ajustada.csv"
unificado.to_csv(out_path, index=False, sep=';', decimal=',', encoding='utf-8')

print("Archivo creado ✅")
print(out_path)
display(unificado.head(10))


Archivo creado ✅
..\..\3_data_final\facturacion_total_ajustada.csv


,FechaEmision,Año,Mes,Imp. Neto Gravado Total,TotalVenta_Real,Tipo
0,2023-02-01,2023,2,1355,12268,Ingreso
1,2023-02-01,2023,2,3966,35907,Ingreso
2,2023-02-01,2023,2,1917,17355,Ingreso
3,2023-02-01,2023,2,1851,16756,Ingreso
4,2023-02-01,2023,2,1528,13839,Ingreso
5,2023-02-01,2023,2,6561,59396,Ingreso
6,2023-02-01,2023,2,2231,20198,Ingreso
7,2023-02-01,2023,2,165,1496,Ingreso
8,2023-02-01,2023,2,1669,15111,Ingreso
9,2023-02-01,2023,2,297,2693,Ingreso
